In [ ]:
## This looked for loading errors up until PTSA load. Was before Cluster though so probably not as useful

In [1]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
HPC_labels = [MTL_labels[i] for i in [0,1,2,3,4,8,9,10,11,12,20,24,25]] # all labels within HPC
ENT_labels = [MTL_labels[i] for i in [6,14,22,26,28]] # all labels within entorhinal
exp = 'FR1'
poss_subs = df[df.experiment == exp]

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/timeseries.py:526: FutureWarning: xarray subclass TimeSeriesX should explicitly define __slots__
  class TimeSeriesX(TimeSeries):


In [22]:
# for now, get an example subject's FR data
# subs = ['R1112M','R1163T'] #'R1112M' # subjects with FR1 and catFR1. SRs of 1000 and 500 tho
# subs = ['R1002P','R1003P','R1006P','R1010J','R1112M','R1163T'] # initial 6 for hippocampus
# subs = ['R1112M','R1018P','R1020J','R1022J','R1023J','R1026D','R1027J'] # 7 with entorhinal
# subs = ['R1010J','R1112M'] # original 2
subs = ['R1345D'] #['R1006P'] #'['R1112M']
# subs = ['R1212P']
bad_subs = ['R1001P''R1015J']
# sess = 2

# sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp)] # all sessions for subs
sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp) & (df.session==1)] # single session
# sub_df = df[(df.experiment == exp) & (df.session == 0)] # all FR subs 
# sub_df = sub_df[4:5]
sub_df

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
1762,NaN,protocols/r1/subjects/R1345D/experiments/FR1/s...,protocols/r1/subjects/R1345D/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1345D/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1345D/localizations/0/m...,NaN,1,R1345D,R1345D,3.1,protocols/r1/subjects/R1345D/experiments/FR1/s...


In [8]:
exp_df = df[df.experiment==exp]
exp_df = exp_df[6:]
exp_df

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
25,NaN,protocols/r1/subjects/R1006P/experiments/FR1/s...,protocols/r1/subjects/R1006P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1006P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1006P/localizations/0/m...,NaN,0,R1006P,R1006P,NaN,protocols/r1/subjects/R1006P/experiments/FR1/s...
26,NaN,protocols/r1/subjects/R1006P/experiments/FR1/s...,protocols/r1/subjects/R1006P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1006P/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1006P/localizations/0/m...,NaN,1,R1006P,R1006P,NaN,protocols/r1/subjects/R1006P/experiments/FR1/s...
37,NaN,protocols/r1/subjects/R1010J/experiments/FR1/s...,protocols/r1/subjects/R1010J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1010J/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1010J/localizations/0/m...,NaN,0,R1010J,R1010J,NaN,protocols/r1/subjects/R1010J/experiments/FR1/s...
38,NaN,protocols/r1/subjects/R1010J/experiments/FR1/s...,protocols/r1/subjects/R1010J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1010J/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1010J/localizations/0/m...,NaN,1,R1010J,R1010J,NaN,protocols/r1/subjects/R1010J/experiments/FR1/s...
48,NaN,protocols/r1/subjects/R1015J/experiments/FR1/s...,protocols/r1/subjects/R1015J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1015J/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1015J/localizations/0/m...,NaN,0,R1015J,R1015J,NaN,protocols/r1/subjects/R1015J/experiments/FR1/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2147,NaN,protocols/r1/subjects/R1462M/experiments/FR1/s...,protocols/r1/subjects/R1462M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1462M/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1462M/localizations/0/m...,NaN,0,R1462M,R1462M,3.4,protocols/r1/subjects/R1462M/experiments/FR1/s...
2149,NaN,protocols/r1/subjects/R1463E/experiments/FR1/s...,protocols/r1/subjects/R1463E/localizations/1/m...,FR1,build,1,protocols/r1/subjects/R1463E/experiments/FR1/s...,1,NaN,0,protocols/r1/subjects/R1463E/localizations/1/m...,NaN,0,R1463E,R1463E_1,3.4,protocols/r1/subjects/R1463E/experiments/FR1/s...
2150,NaN,protocols/r1/subjects/R1463E/experiments/FR1/s...,protocols/r1/subjects/R1463E/localizations/1/m...,FR1,build,1,protocols/r1/subjects/R1463E/experiments/FR1/s...,1,NaN,1,protocols/r1/subjects/R1463E/localizations/1/m...,NaN,1,R1463E,R1463E_1,3.4,protocols/r1/subjects/R1463E/experiments/FR1/s...
2164,NaN,protocols/r1/subjects/R1466J/experiments/FR1/s...,protocols/r1/subjects/R1466J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1466J/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1466J/localizations/0/m...,NaN,0,R1466J,R1466J,3.4,protocols/r1/subjects/R1466J/experiments/FR1/s...


## TEST HOW MANY SESSIONS RUN AND LOG THOSE THAT DON'T

In [4]:
time_add_save

[]

In [ ]:
import mne
from ptsa.data.filters import ButterworthFilter
from scipy.signal import firwin,filtfilt,kaiserord
%autoreload

save_values = 0
filter_type = 'hamming'#'hamming'# 'butter' # Vaz used Butterworth from 80-120 Hz; Norman used Hamming from 70-180 Hz
# •Butterworth from Vaz et al: 2nd order from 80-120 ms, hilbert, select events >2 STD above mean of filtered traces.
#      Events >=25 ms long w/ max amp >3 SD were retained as ripples. Join adjacent ripples separated by <15 ms 
# •Hilbert from Norman et al: "70-180 Hz zero-lag linear-phase Hamming FIR filter w/ transition bandwidth of 5 Hz"
#      Then Hilbert, clip extreme to 4 SD, square this clipped, smooth w/ Kaiser FIR low-pass filter with 40 Hz cutoff,
#      mean and SD computed across entire experimental duration to define the threshold for event detection
#      Events from original (squared but unclipped) signal >4 SD above baseline were selected as candidate SWR events. 
#      Duration expanded until ripple power <2 SD. Events <20 ms or >200 ms excluded. Adjacent events <30 ms separation (peak-to-peak) merged.
# •Using IED detection from Vaz for 250 Hz highpass events (really 250-499). Norman uses 20-60 Hz events which is...odd.
#       See detectRipples code in module for this algorithm

selected_region = HPC_labels #HPC_labels # ENT_labels+HPC_labels 

# recall
IRI = 2000 # inter-ripple interval...remove ripples within this range (keep only first one and remove those after it)
psth_start = -IRI # only makes sense to look at period <= IRI
psth_end = 2000 # how long to grab data after recall
retrieval_whole_time = 30000

# encoding
encoding_time = 2000 # actual preentation is 1.6 s + 0.75-1.0 s so just keep 2.0 s of data for PSTH
pre_encoding_time = -500 # since minimum ISI is 0.75 s let's only plot the 500 ms before word on

desired_sample_rate = 500. # in Hz. This seems like lowerst common denominator recording freq.

if 'entorhinal' in selected_region and 'left hippocampus' in selected_region:
    region_name = 'HPC_ENT'
elif 'entorhinal' in selected_region:
    region_name = 'ENT'
elif 'left hippocampus' in selected_region:
    region_name = 'HPC'
ripple_array = []; ripple_array2 = []; fr_array = []; ripple_array_enc = []
trial_nums = []; trial_nums2 = []; trial_nums_enc = []; encoded_word_key_array = []
session_ct = 0; channel_ct = 0
HPC_names = []; sub_names = []; sub_sess_names = []
electrodes_per_session = []
total_lists = 0; total_recalls = 0; kept_recalls = 0
ent_elec_ct = []; sd_regions = []; not_sd_regions = []
ripple_ied_accum_ct = []
time_add_save = []

for row in exp_df.itertuples(): #sub_df.itertuples():   
    try:
        sub = row.subject; session = row.session; exp = row.experiment
        mont = int(row.montage); loc = int(row.localization)
        reader = CMLReadDFRow(row)
        evs = reader.load('task_events')
        evs_free_recall = evs[(evs.type=='REC_WORD') & (evs.recalled==True)] # recalled word AND correct (in previous list)
        total_recalls = total_recalls + len(evs_free_recall) # get total recalls from lists
        total_lists = total_lists + len(evs[evs.type=='WORD'].list.unique()) # get total lists

        # remove events with Inter-Recall Intervals too small. Use psth_start for IRI since that's what will show in PSTH
        evs_free_recall = evs_free_recall[np.append(True,np.diff(evs_free_recall.mstime)>IRI)] 
        kept_recalls = kept_recalls + len(evs_free_recall)

        # I'm going to save encoding word events too, but need a mask to keep track of:
        # 0) words not recalled 1) words recalled from this list 2) words later recalled BUT IRI<2 s so removed
        evs_encoding_words = evs[evs.type=='WORD']
        encoded_word_key = np.zeros(len(evs_encoding_words)) # 0 for not recalled
        encoded_word_key[evs_encoding_words.recalled==True] = 2 # 2 for recalled but removed bc IRI<2 s
        encoded_word_key[evs_encoding_words.item_name.isin(evs_free_recall.item_name.unique())] = 1 # recalled words
        # since finding all encoding words IN the list of correctly free recalled words won't have any intrusions

        pairs = reader.load('pairs')
        tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
        elec_regions,atlas_type = get_elec_regions(tal_struct) 
    #     # if you want to check regions labeled by stein/das v. those that weren't
    #     sd_regions = np.append(sd_regions,elec_regions[atlas_type=='stein'])
    #     sd_regions = np.append(sd_regions,elec_regions[atlas_type=='das'])
    #     not_sd_regions = np.append(not_sd_regions,elec_regions[(atlas_type!='stein') & (atlas_type!='das')])

        # also let's grab whole retrieval periods for a better baseline of SWRs
        evs_rets = evs[evs.type=='REC_START']
        # count elec regions with specific string...mostly here to comment out below and test for good sessions
        if len(findAinB(ENT_labels,elec_regions))>0:
            ent_ct = len(findAinB(ENT_labels,elec_regions))
            ent_elec_ct = np.append(ent_elec_ct,sub+'_'+str(session)+'_ct-'+str(ent_ct))

        # load eeg (events X electrodes X time)
        eeg = reader.load_eeg(events=evs_free_recall, rel_start=psth_start, rel_stop=psth_end, clean=True, scheme=pairs)
        print('loaded eeg for '+sub+str(session))
        sr = eeg.samplerate
    
        # if weird samplerate, add a few ms to make the load work
        if (499<sr<500) | (998<sr<1000):
            time_add = 1
            if (499<sr<500):
                sr = 500
            elif (998<sr<1000):
                sr = 1000
            while eeg.shape[2] < (psth_end-psth_start)/(1000/sr):
                eeg = reader.load_eeg(events=evs_free_recall, rel_start=psth_start, rel_stop=psth_end+time_add, clean=True, scheme=pairs)
                if time_add>50:
                    break
                time_add+=1
            time_add_save.append(time_add)
            if eeg.shape[2] not in [2000,4000]:
                e = 'Manual error: adding +1 did not help here with load_eeg, sr = '+str(sr)
                LogDFExceptionLine(row, e, 'SWR_get_eeg_log2.txt')
            
#         good_srs = [500,1000,1600,2000]        
#         if sr not in good_srs:
#             e = 'Sampling rate is atypical, sr = '+str(sr)
#             LogDFExceptionLine(row, e, 'SWR_get_eeg_log2.txt')
#         if np.mod(sr,1)>0: # if not an integer, there are issues, so skip. E.g. R1006P, session 1 has sr=499.7
#             print('SR was '+str(sr)+' so skip session')
#             continue
#         else:
#             sr = int(sr)
#             print('Sampling rate: '+str(sr))
        print('getting PTSA')
        eeg_ptsa = eeg.to_ptsa()
#         # grabbing whole retrieval periods to look at those
#         eeg_ptsa2 = reader.load_eeg(events=evs_rets, rel_start=psth_start, 
#                                     rel_stop=psth_end+retrieval_whole_time, clean=True, scheme=pairs).to_ptsa()
#         # grabbing encoding period too now
#         eeg_ptsa_enc = reader.load_eeg(events=evs_encoding_words, rel_start=pre_encoding_time,
#                                     rel_stop=encoding_time, clean=True, scheme=pairs).to_ptsa()

#         ## FILTERS ##
        trans_width = 5.
        ntaps = (2/3)*np.log10(1/(10*(1e-3*1e-4)))*(sr/trans_width) # gives 400 with sr=500, trans=5
        FIR_bandstop = firwin(int(ntaps+1), [70.,180.], fs=sr, window='hamming',pass_zero='bandstop')
#     #     FIR_bandpass = firwin(int(ntaps), [70.,180.], fs=sr, window='hamming',pass_zero='bandpass')
#         bandstop_25_60 = firwin(int(ntaps+1), [20.,60.], fs=sr, window='hamming',pass_zero='bandstop') # Norman 2019 IED
#     #     bandstop_60_80 = firwin(int(ntaps+1), [60.,80.], fs=sr, window='hamming',pass_zero='bandstop') # Gelinas 2016 IED
#         nyquist = sr/2
#         ntaps40, beta40 = kaiserord(40, trans_width/nyquist)
#         kaiser_40lp_filter = firwin(ntaps40, cutoff=40, window=('kaiser', beta40), scale=False, nyq=nyquist, pass_zero='lowpass')

        # filter for ripples using filter selected above
        if filter_type == 'hamming':
            # need to subtract out to get the filtered signal since default is bandstop but want to keep it as PTSA        
            eeg_rip_band = eeg_ptsa-filtfilt(FIR_bandstop,1.,eeg_ptsa) 
#             eeg_rip_band2 = eeg_ptsa2-filtfilt(FIR_bandstop,1.,eeg_ptsa2)
#             #         eeg_rip_band = filtfilt(FIR_bandpass,1.,eeg_ptsa) # can't use ptsa_to_mne this way so use eeg-bandstopped signal
#             eeg_ied_band = eeg_ptsa-filtfilt(bandstop_25_60,1.,eeg_ptsa)
#             eeg_ied_band2 = eeg_ptsa2-filtfilt(bandstop_25_60,1.,eeg_ptsa2)  
#             eeg_rip_band_enc = eeg_ptsa_enc-filtfilt(FIR_bandstop,1.,eeg_ptsa_enc) 
#             eeg_ied_band_enc = eeg_ptsa_enc-filtfilt(bandstop_25_60,1.,eeg_ptsa_enc)
#         elif filter_type == 'butter':
#             eeg_rip_band = ButterworthFilter(timeseries=eeg_ptsa, freq_range=[80.,120.], filt_type='bandpass',order=2).filter()
#             eeg_rip_band2 = ButterworthFilter(timeseries=eeg_ptsa2, freq_range=[80.,120.], filt_type='bandpass',order=2).filter()
#             if sr == 500: # dropped below 250 Hz because too close Nyquist
#                 eeg_ied_band = ButterworthFilter(timeseries=eeg_ptsa, freq_range=200., filt_type='highpass',order=2).filter() 
#                 eeg_ied_band2 = ButterworthFilter(timeseries=eeg_ptsa2, freq_range=200., filt_type='highpass',order=2).filter() 
#             elif sr >= 1000:    
#                 #this seems okay since large range far from Nyquist...
#                 eeg_ied_band = ButterworthFilter(timeseries=eeg_ptsa, freq_range=[250.,499.], filt_type='bandpass',order=2).filter()
#                 eeg_ied_band2 = ButterworthFilter(timeseries=eeg_ptsa2, freq_range=[250.,499.], filt_type='bandpass',order=2).filter()
#             eeg_raw = ptsa_to_mne(eeg_ptsa,[0,psth_end-psth_start]) 
#             eeg_raw2 = ptsa_to_mne(eeg_ptsa2,[0,retrieval_whole_time+(psth_end-psth_start)])     
        eeg_rip_band = ptsa_to_mne(eeg_rip_band,[0,psth_end-psth_start])
        if (eeg_rip_band.get_data().shape[0] != len(evs_free_recall)):
            e = 'Manual error: MNE eeg evs do not match len(evs_free_recall)'
            LogDFExceptionLine(row, e, 'SWR_get_eeg_log2.txt')        
        
#         eeg_ied_band = ptsa_to_mne(eeg_ied_band,[0,psth_end-psth_start])
#         _ = eeg_ied_band.apply_hilbert(envelope=True)  

#         eeg_rip_band2 = ptsa_to_mne(eeg_rip_band2,[0,retrieval_whole_time+(psth_end-psth_start)])
#         _ = eeg_rip_band2.apply_hilbert(envelope=True)
#         eeg_ied_band2 = ptsa_to_mne(eeg_ied_band2,[0,retrieval_whole_time+(psth_end-psth_start)])
#         _ = eeg_ied_band2.apply_hilbert(envelope=True)

#         eeg_rip_band_enc = ptsa_to_mne(eeg_rip_band_enc,[0,encoding_time-pre_encoding_time])
#         _ = eeg_rip_band_enc.apply_hilbert(envelope=True)
#         eeg_ied_band_enc = ptsa_to_mne(eeg_ied_band_enc,[0,encoding_time-pre_encoding_time])    
#         _ = eeg_ied_band_enc.apply_hilbert(envelope=True)

#         HPC_session_ct = 0
#         for channel in range(len(elec_regions)):
#             if elec_regions[channel] in selected_region:
#                 channel_ct+=1 # total channels from beginning 
#                 HPC_session_ct+=1 # channel ct for this session
#                 HPC_names.append(elec_regions[channel])
#                 sub_names.append(sub)
#                 sub_sess_names.append(sub+'-'+str(session))
#                 encoded_word_key_array.append(encoded_word_key) # save the key for each electrode so easier to unpack later

#                 # first, make a matrix of the times of free recall
#                 list_nums = evs_rets.list.unique()
#                 temp_fr_array = np.zeros((len(list_nums),retrieval_whole_time))
#                 for ln in list_nums:
#                     list_times = evs_free_recall[evs_free_recall.list==ln].rectime # I confirmed these are accurate relative to REC_START
#                     temp_fr_array[ln-1][np.array(list_times)] = 1            
#                 fr_array = superVstack(fr_array,temp_fr_array)

#                 # eeg
#                 eeg_rip = eeg_rip_band.get_data()[:,channel,:] # get data from MNE container
#                 eeg_rip2 = eeg_rip_band2.get_data()[:,channel,:]     
#                 eeg_ied = eeg_ied_band.get_data()[:,channel,:]
#                 eeg_ied2 = eeg_ied_band2.get_data()[:,channel,:]
#                 eeg_rip_enc = eeg_rip_band_enc.get_data()[:,channel,:]
#                 eeg_ied_enc = eeg_ied_band_enc.get_data()[:,channel,:]

#                 # select detection algorithm (note that iedlogic is same for both so always run that)

#                 if filter_type == 'hamming':
#                     # filter IEDs 
#                     eeg_ied = eeg_ied**2 # already rectified now square
#                     eeg_ied2 = eeg_ied2**2  
#                     eeg_ied_enc = eeg_ied_enc**2
#                     eeg_ied = filtfilt(kaiser_40lp_filter,1.,eeg_ied) # low pass filter
#                     eeg_ied2 = filtfilt(kaiser_40lp_filter,1.,eeg_ied2)         
#                     eeg_ied_enc = filtfilt(kaiser_40lp_filter,1.,eeg_ied_enc)   
#                     mean1 = np.mean(eeg_ied)
#                     std1 = np.std(eeg_ied)
#                     iedlogic = eeg_ied>=mean1+4*std1 # Norman et al 2016
#                     mean2 = np.mean(eeg_ied2)
#                     std2 = np.std(eeg_ied2)            
#                     iedlogic2 = eeg_ied2>=mean2+4*std2
#                     mean_enc = np.mean(eeg_ied_enc)
#                     std_enc = np.std(eeg_ied_enc)            
#                     iedlogic_enc = eeg_ied_enc>=mean_enc+4*std_enc                
#                     # detect ripples
#                     ripplelogic = detectRipplesHamming(eeg_rip,trans_width,sr,iedlogic)
#                     ripplelogic2 = detectRipplesHamming(eeg_rip2,trans_width,sr,iedlogic2)
#                     ripplelogic_enc = detectRipplesHamming(eeg_rip_enc,trans_width,sr,iedlogic_enc)
#                 elif filter_type == 'butter':
#                     eeg_mne = eeg_raw.get_data()[:,channel,:]
#                     eeg_mne2 = eeg_raw2.get_data()[:,channel,:]
#                     # detect ripples
#                     ripplelogic, iedlogic = detectRipplesButter(eeg_rip,eeg_ied,eeg_mne,sr)
#                     ripplelogic2, iedlogic2 = detectRipplesButter(eeg_rip2,eeg_ied2,eeg_mne2,sr)               

#                 ripple_ied_accum_ct.append([sub,session,sr,sum(ripplelogic),sum(iedlogic),sum(ripplelogic2),sum(iedlogic2),
#                                             sum(ripplelogic_enc),sum(iedlogic_enc)])

#                 # have to make a decision on upsampling v. downsampling since there are diff. sampling rates
#                 if sr>desired_sample_rate: # downsampling here for anything greater than 500...will be issues if SR not divisible
#                     # but can fix that in the future if necessary (maybe can FFT resample?)
#                     ripplelogic = downsampleBinary(ripplelogic,sr/desired_sample_rate)
#                     ripplelogic2 = downsampleBinary(ripplelogic2,sr/desired_sample_rate)
#                     ripplelogic_enc = downsampleBinary(ripplelogic_enc,sr/desired_sample_rate)
#     #                 iedlogic = downsampleBinary(iedlogic,sr/desired_sample_rate)                
#     #                 iedlogic2 = downsampleBinary(iedlogic2,sr/desired_sample_rate)
#                 ripple_array = superVstack(ripple_array,ripplelogic) # append across electrodes
#                 ripple_array2 = superVstack(ripple_array2,ripplelogic2) # append across electrodes   
#                 ripple_array_enc = superVstack(ripple_array_enc,ripplelogic_enc) # append across electrodes   
#                 print('Cumulative array size: ')
#                 print(ripple_array.shape)
#                 trial_nums = np.append(trial_nums,ripplelogic.shape[0])
#                 trial_nums2 = np.append(trial_nums2,ripplelogic2.shape[0])
#                 trial_nums_enc = np.append(trial_nums_enc,ripplelogic_enc.shape[0])
#         electrodes_per_session = np.append(electrodes_per_session,HPC_session_ct)
#         session_ct+=1
        
    except Exception as e:
        LogDFExceptionLine(row, e, 'SWR_get_eeg_log2.txt') #np.save('SWRscript_FR1.npy', ['Exception', str(e.__class__), str(e)])
        continue

# csvWriter(ripple_ied_accum_ct,'ieds') # if you want to save the accumulated ripple numbers

if save_values == 1:
    if session_ct == 1:
        fn = os.path.join('/home1/john/SWR/',
            'SWR_'+region_name+'_from-'+str(session_ct)+'_sessions_in-'+str(len(np.unique(sub_names)))+'subjects_'+filter_type+'_'+sub+'.p')            
    else:
        fn = os.path.join('/home1/john/SWR/',
            'SWR_'+region_name+'_from-'+str(session_ct)+'_sessions_in-'+str(len(np.unique(sub_names)))+'subjects_'+filter_type+'.p')            
    with open(fn,'wb') as f:
        pickle.dump({'channel_ct':channel_ct, 'session_ct':session_ct,
                    'HPC_names':HPC_names, 'sub_sess_names':sub_sess_names,
                    'ripple_array':ripple_array, 'ripple_array2':ripple_array2, 'ripple_array_enc':ripple_array_enc,
                    'trial_nums':trial_nums, 'trial_nums2':trial_nums2,'trial_nums_enc':trial_nums_enc,
                    'fr_array':fr_array, 'sub_names':sub_names, 'encoded_word_key_array':encoded_word_key_array,
                    'total_recalls':total_recalls, 'kept_recalls':kept_recalls}, f)


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1006P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1006P/tal/R1006P_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1006P/experiments/FR1/sessions/0/ephys/current_processed/sources.json
/data/eeg/R1006P/eeg.noreref/params.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)


loaded eeg for R1006P0
getting PTSA
64 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1006P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1006P/tal/R1006P_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1006P/experiments/FR1/sessions/1/ephys/current_processed/sources.json
/data/eeg/R1006P/eeg.noreref/R1006P_12Feb15_1126.params.txt
/data/eeg/R1006P/eeg.noreref/params.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)


loaded eeg for R1006P1
getting PTSA
33 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1010J/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1010J/tal/R1010J_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1010J/experiments/FR1/sessions/0/ephys/current_processed/sources.json
/data/eeg/R1010J/eeg.noreref/params.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)


...

loaded eeg for R1351M0
getting PTSA
41 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1354E/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1354E/tal/R1354E_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/readers/eeg.py:377: MissingChannelsWarning: The following channels are missing: 5Ldm1-5Ldm2, 5Ldm2-5Ldm3, 5Ldm3-5Ldm4, 5Ldm4-5Ldm5, 5Ldm5-5Ldm6, 5Ldm6-5Ldm7, 5Ldm7-5Ldm8, 5Ldm8-5Ldm9, 5Ldm9-5Ldm1, 25Ldm1-25Ldm2, 25Ldm2-25Ldm3, 25Ldm3-25Ldm4, 25Ldm4-25Ldm5, 25Ldm5-25Ldm6, 25Ldm6-25Ldm7, 25Ldm7-25Ldm8, 25Ldm8-25Ldm9, 25Ldm9-25Ldm1
  warnings.warn(msg, MissingChannelsWarning)


loaded eeg for R1354E0
getting PTSA
58 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
loaded eeg for R1354E1
getting PTSA
51 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1355T/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1355T/tal/R1355T_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)


loaded eeg for R1355T0
getting PTSA
47 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
loaded eeg for R1355T1
getting PTSA
23 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
loaded eeg for R1355T2
getting PTSA
28 matching events found
No baseline correction applied
Not setting metadata


In [8]:
len(tal_struct)
sub
session

110

'R1063C'

1